In [ ]:
#pip install tweet-preprocessor
import pandas as pd
import preprocessor as p
import re
import string

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
class TextProcess(object):
    def cleanText(self, text):
        text = self.removeWebItems(text)
        text = self.removeNumbers(text)
        text = self.removeSpaces(text)
        text = self.cleanPunctuation(text)
        return text
        
    #remve Punctuation and convert all text to lower case
    def cleanPunctuation(self, text): 
        return ''.join(char for char in text if char not in string.punctuation).lower()

    #remove more than one space
    def removeSpaces(self, text):
        return re.sub('\s+', ' ', text).strip()

    # remove numbers
    def removeNumbers(self, text):
        return re.sub(r'\d+', '', text)
    
    #URLs, Hashtags,Mentions, Reserved words (RT, FAV), Emojis, Smileys
    def removeWebItems(self, text):
        return p.clean(text)

In [ ]:
userTweets = pd.read_csv("tweetsData.csv")
userTweets.columns = ['name','lang','date','tweets','generation']
clean = TextProcess()
userTweets['cleanedTweetText'] = userTweets['tweets'].apply(lambda tweets: clean.cleanText(tweets))

In [ ]:
# remove empty tweets after clean up
cleanUserTweets = userTweets[userTweets['cleanedTweetText'] != '']

In [ ]:
dataSet = cleanUserTweets[['generation', 'cleanedTweetText']]
dataSet = dataSet.drop_duplicates()

In [ ]:
columns = "word 30s 40s 50s 60s 70s 80s 90s 00s".split()
wordMatrix = pd.DataFrame([], columns=columns)
wordMatrix = wordMatrix.set_index(['word'])
datax = dataSet.head()

In [ ]:
for index, row in datax.iterrows():
    text = row['cleanedTweetText']
    generation = row['generation']
    for word in text.split():
        if word not in wordMatrix.index.values:
            wordMatrix.loc[word] = [0, 0, 0, 0, 0, 0, 0, 0]
        wordMatrix.loc[word,generation] = wordMatrix.loc[word,generation] + 1
    
wordMatrix

In [ ]:
generation = "30s 40s 50s 60s 70s 80s 90s 00s".split()
x = wordMatrix[generation]
y = wordMatrix.index

In [ ]:
#split the data to traning and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
regression = LogisticRegression()
#train the data
regression.fit(x_train, y_train)
#test the data
train_score = regression.score(x_train, y_train)
test_score = regression.score(x_test, y_test)
print('Training accuracy: '+str(train_score))
print('Testing accuracy: '+str(test_score))